In [1]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_io, image_render
import skimage.exposure

In [2]:
# Paths
pca_data_model_file = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_PCA_Model/20210825_OlbergAreaA_PCA_data_and_model.npz')
hyspec_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4_Rad_Georef_SGC')
pca_im_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5_Rad_Georef_SGC_PCA')

In [3]:
# Load PCA model
with np.load(pca_data_model_file) as npz_files:
    X_mean = npz_files['spectra_mean']
    X_std = npz_files['spectra_std']
    W_pca = npz_files['pca_components']

In [4]:
hyspec_images = [file for file in hyspec_dir.glob('*.hdr')]

In [5]:
for hyspec_image_path in tqdm.tqdm(hyspec_images):
    # Load image
    image,wl,rgb_ind,_ = hyspec_io.load_envi_image(str(hyspec_image_path))
    
    # Create PCA image
    im_vec = np.reshape(image,(-1,image.shape[-1]))
    im_vec_norm = (im_vec-X_mean)/X_std
    im_vec_pca = im_vec_norm @ W_pca
    im_pca = np.reshape(im_vec_pca,image.shape[0:2]+(im_vec_pca.shape[-1],))
    
    # Percentile stretch image and show 3 first components as RGB
    nonzero_mask = ~np.all(image==0,axis=2,keepdims=True)
    im_pca_rgb = image_render.percentile_stretch(im_pca[:,:,[2,1,0]])*nonzero_mask

    # Save PCA image to file
    pca_file = pca_im_dir / (str(hyspec_image_path.name).split('.')[0] + '.png')

100%|██████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.97s/it]
